In [ ]:
import numpy as np
import cv2 as cv
img = cv.imread('/home/socmgr/red.jpg')
b,g,r = cv.split(img)
img = cv.merge((b,g,r))
img2 = img[:,:,0]
#  0->블루 1->그린 2->레드 인데 0으로 해서 블루채널에 해당하는 이미지출력
img[:,:,2] = 0
# 3번째 꺼 0->블루 1->그린 2->레드 인데 0으로하면 해당채널색깔 빼고 이미지출력 
cv.imshow('image',img)
cv.imshow('image2',img2)
cv.imshow('blue', b)
cv.imshow('green', g)
cv.imshow('red', r)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from skimage.exposure import rescale_intensity

def convolve(image, kernel, pad, stride):
	# grab the spatial dimensions of the image, along with
	# the spatial dimensions of the kernel
	(iH, iW) = image.shape[:2]
	(kH, kW) = kernel.shape[:2]
	# allocate memory for the output image, taking care to
	# "pad" the borders of the input image so the spatial
	# size (i.e., width and height) are not reduced
	b,g,r = cv.split(img)
	b = cv.copyMakeBorder(b, pad, pad, pad, pad, 0)
	g = cv.copyMakeBorder(g, pad, pad, pad, pad, 0)
	r = cv.copyMakeBorder(r, pad, pad, pad, pad, 0)
	padding = int((kW - 1) / 2)
	outputb = np.zeros((int((iH+pad*2-kH)/stride+1), int((iW+pad*2-kW)/stride+1)), dtype="float32")
	outputg = np.zeros((int((iH+pad*2-kH)/stride+1), int((iW+pad*2-kW)/stride+1)), dtype="float32")
	outputr = np.zeros((int((iH+pad*2-kH)/stride+1), int((iW+pad*2-kW)/stride+1)), dtype="float32")    
	# loop over the input image, "sliding" the kernel across
	# each (x, y)-coordinate from left-to-right and top to
	# bottom
	for y in np.arange(padding, int((iH+pad*2-kH)/stride+1)):
		for x in np.arange(padding, int((iW+pad*2-kW)/stride+1)):
			# extract the ROI of the image by extracting the
			# *center* region of the current (x, y)-coordinates
			# dimensions
			roib = b[int((stride*y) - padding):int((stride*y)+ padding + 1), 
                        int((stride*x) - padding):int((stride*x)+ padding + 1)]
			roig = g[int((stride*y) - padding):int((stride*y)+ padding + 1), 
                        int((stride*x) - padding):int((stride*x)+ padding + 1)]  
			roir = r[int((stride*y) - padding):int((stride*y)+ padding + 1), 
                        int((stride*x) - padding):int((stride*x)+ padding + 1)]  
			# perform the actual convolution by taking the
			# element-wise multiplicate between the ROI and
			# the kernel, then summing the matrix
			kb = (roib * kernel).sum()
			kg = (roig * kernel).sum()
			kr = (roir * kernel).sum()            
			# store the convolved value in the output (x,y)-
			# coordinate of the output image

			outputb[y - padding, x - padding] = kb
			outputg[y - padding, x - padding] = kg
			outputr[y - padding, x - padding] = kr            
        	# rescale the output image to be in the range [0, 255]
	outputb = rescale_intensity(outputb, in_range=(0, 255))
	outputb = (outputb * 255).astype("uint8")
	outputg = rescale_intensity(outputg, in_range=(0, 255))
	outputg = (outputg * 255).astype("uint8")
	outputr = rescale_intensity(outputr, in_range=(0, 255))
	outputr = (outputr * 255).astype("uint8")
	output = cv.merge((outputb,outputg,outputr))
 
	# return the output image
	return output
img = cv.imread('/home/socmgr/red.jpg')
#kernel = np.ones((5,5),np.float32)/25  ->>averaging blurring kernel
kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]) # 파이썬에서 np를 써서 배열을 만들면 성능이 더 좋아짐 
kernel2 = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
dst = convolve(img,kernel,1,1)
dst2 = convolve(img,kernel2,50,2)
dst3 = cv.filter2D(img,-1,kernel)
cv.imshow('img',dst)
cv.imshow('img2',img)
cv.imshow('img3', dst2)
cv.imshow('img4', dst3)
cv.waitKey(0)
cv.destroyAllWindows()

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from skimage.exposure import rescale_intensity

def convolve(image, kernel):
    # grab the spatial dimensions of the image, along with
    # the spatial dimensions of the kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
 
    # allocate memory for the output image, taking care to
    # "pad" the borders of the input image so the spatial
    # size (i.e., width and height) are not reduced
    pad = int((kW - 1) / 2)
    image = cv.copyMakeBorder(image, pad, pad, pad, pad, cv.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float32")
        # loop over the input image, "sliding" the kernel across
    # each (x, y)-coordinate from left-to-right and top to
    # bottom
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # extract the ROI of the image by extracting the
            # *center* region of the current (x, y)-coordinates
            # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            # perform the actual convolution by taking the
            # element-wise multiplicate between the ROI and
            # the kernel, then summing the matrix
            k = (roi * kernel).sum()
 
            # store the convolved value in the output (x,y)-
            # coordinate of the output image
            output[y - pad, x - pad] = k
                # rescale the output image to be in the range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
 
    # return the output image
    return output

img = cv.imread('/home/socmgr/red.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#kernel = np.ones((5,5),np.float32)/25  ->>averaging blurring kernel
kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]) # 파이썬에서 np를 써서 배열을 만들면 성능이 더 좋아짐
kernel2 = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
dst = convolve(gray,kernel)
dst2 = convolve(gray,kernel2)
cv.imshow('img',dst)
cv.imshow('img2',gray)
cv.imshow('img3', dst2)
cv.waitKey(0)
cv.destroyAllWindows()    